# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: Visualisierungen der gemessenen Sensordaten

Hier in diesem Notebook visualisieren wir alle gemessenen Sensordaten und versuchen so zu identifizieren, welche aufgenommenen Sensordaten von relevanz sein können bzw. welche widerum nicht. 

In [1]:
# Laden von Libraries
%pip install -r requirements.txt
from data_ingestion.pipeline_manager import PipelineRunner as PipelineRunner 
from data_ingestion import utilities as util
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import seaborn as snsb
import polars as pl

You should consider upgrading via the 'c:\Users\Si Ben Tran\venv\cdl1\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


## Import von Daten

- Einlesen der Daten mittels Pipeline oder
- Einlesen der gefilterten Daten von 'all-sensormeasurement-plots'

In [2]:
# Daten einlesen und Ausgeben 
path = "Messungen"
pipeline = None
origin = "SensorLogger"

runner = PipelineRunner(path)
runner.set_origin(origin)
runner.set_pipeline(pipeline)
data = runner.run()
path: pathlib.Path = "Export.arrow"
data.write_ipc(path)
data

Read Files:   0%|          | 0/210 [00:00<?, ?files/s]

time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,id,user,class
datetime[ms],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
2023-02-27 15:02:04.548,0.0,-3.552,-9.14,null,null,null,null,null,null,0.161765,-0.08829,-0.470671,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:17.156,0.0,9.722,1.278,null,null,null,null,null,null,-0.21775,-0.719579,0.631111,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:31.574,-0.728,9.779,0.0,null,null,null,null,null,null,-0.42065,0.568356,-0.598058,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:16.685,null,null,null,null,null,null,0.0,43.5,-13.14375,null,null,null,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:32.295,null,null,null,null,null,null,-13.012501,0.0,20.587502,null,null,null,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:33.285,null,null,null,null,null,null,-22.612501,0.0,20.268751,null,null,null,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:35.195,null,null,null,null,null,null,-12.1125,0.0,20.75625,null,null,null,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:45.315,null,null,null,null,null,null,-21.243752,0.0,16.9125,null,null,null,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
2023-02-27 15:02:48.985,null,null,null,null,null,null,-41.100002,0.0,-3.24375,null,null,null,"""01_SamsungA22-...","""Ben_Tran""","""Laufen"""
